# Transform raw extract to dense representation of non-static medical features

In [ ]:
'''
This code is modification of:
https://github.com/StatNLP/mlhc_2024_prediction_of_causes/blob/main/eicu_experiments/eicu_03_pickle2dense.py
The necessary inputs for this script are created via:
https://github.com/StatNLP/mlhc_2024_prediction_of_causes/blob/main/mimic_experiments/mimic3_01_preprocess_icu.py
'''

In [ ]:
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm

In [ ]:
def inv_list(l, start=0):  # Create: var:ind mappings
    d = {}
    for i in range(len(l)):
        d[l[i]] = i+start
    return d

#f create masked outputs
def f(x):
    mask   = [0 for i in range(V)]
    values = [0 for i in range(V)]
    for vv in x:  # tuple of ['vind','value']
        v = int(vv[0])-1  # shift index of vind
        mask[v] = 1
        values[v] = vv[1]  # get value
    return values+mask  # concat

In [ ]:
dataset_name = "mimic-short"

In [ ]:
data_path = '<location of pickle file produced by mimic3_02_preprocess_pickle.py>'

data, _ , train_ind, valid_ind, test_ind = pickle.load(open(data_path, 'rb'))
del _

In [ ]:
#calculate mean, sd for studentization
means_stds = data.groupby("variable").agg({"mean":"first", "std":"first"})

mean_std_dict = dict()
for pos, row in means_stds.iterrows():
    mean_std_dict[pos] = (float(row["mean"]), float(row["std"]))

with open('../gold_data/'+ dataset_name +'/mean_std_dict.pickle', "wb") as outfile:
    pickle.dump(mean_std_dict, outfile)

In [ ]:
#reduce data so that it contains only ts occuring in train, valid or test
data = data.loc[data.ts_ind.isin(np.concatenate((train_ind, valid_ind, test_ind), axis=-1))]

#Select first 48h hours of each time series.
data = data.loc[(data.hour >= 0) & (data.hour <= 48)]

# Fix age.
data.loc[(data.variable == 'Age') & (data.value > 200), 'value'] = 91.4

#fix index data type
data['ts_ind'] = data['ts_ind'].astype(int)

#In order to calculate N (number of time series) correctly this way. ts_ind must start with 0 and gap less 
#ALTERNATIVE: N=data['ts_int'].nunique() 
N = int(data.ts_ind.max() + 1)

# remove static variables from data
data = data.loc[~data.variable.isin(['Age', 'Gender'])]

#number of non-static variables extracted from mimic-db by the previous script. It' important for the mapping and the emedding model
V = data['variable'].nunique()

In [ ]:
#generate a dense array version of non-static data
varis = sorted(list(set(data.variable)))
var_to_ind = inv_list(varis, start=1)

#add numeric indentifier for variable names to data
data['vind'] = data.variable.map(var_to_ind)
#reduce columns of data and introduce sorting: zeitreihe|variable|zeit
data = data[['ts_ind', 'vind', 'hour', 'value']].sort_values(by=['ts_ind', 'vind', 'hour'])
#add an oberservation identifier to each time series starting with 0 and incrementing by 1
data = data.sort_values(by=['ts_ind']).reset_index(drop=True)
data = data.reset_index().rename(columns={'index': 'obs_ind'})
data = data.merge(data.groupby('ts_ind').agg({'obs_ind': 'min'}).reset_index().rename(columns={'obs_ind': 'first_obs_ind'}), on='ts_ind')
data['obs_ind'] = data['obs_ind'] - data['first_obs_ind']


#After this step we have a densification of the first 48h of for each time series
for pred_window in tqdm(range(0, 48, 1)):
    #selection of 1h time window. Remark: to non-sharp unequal signs
    pred_data = data.loc[(data.hour >= (w + pred_window)) & (data.hour <= (w + pred_window + 1))] 
    #pick first observed measurment within that window
    pred_data = pred_data.groupby(['ts_ind', 'vind']).agg({'value': 'first'}).reset_index() 
    #MH: cast var_id an value to a list and add 
    pred_data['vind_value' + str(pred_window)] = pred_data[['vind', 'value']].values.tolist() 
    pred_data = pred_data.groupby('ts_ind').agg({'vind_value' + str(pred_window): list}).reset_index()
    # turn to dense representation with mask
    pred_data['vind_value' + str(pred_window)] = pred_data['vind_value' + str(pred_window)].apply(f)
    #add variables 'vind_value<0-47>' storing the dense vectors
    if pred_window == 0:
        obs_data = pred_data
    else:
        obs_data = obs_data.merge(pred_data, on = 'ts_ind') 


#change obs_data so that for each ts obs_ind starts with 0
obs_data = obs_data.sort_values(by=['ts_ind']).reset_index(drop=True)
obs_data = obs_data.reset_index().rename(columns={'index': 'obs_ind'})
obs_data = obs_data.merge(data.groupby('ts_ind').agg({'obs_ind': 'min'}).reset_index().rename(columns={'obs_ind': 'first_obs_ind'}), on='ts_ind')
obs_data['obs_ind'] = obs_data['obs_ind'] - obs_data['first_obs_ind']


#(1) list(obs_data['vind_valueX']) is a list with #ts length of list with length 262
#(2) [... for pred_windows in range(0,48)] creates a list of those lists 
#so blub is a np array (48, #ts, 262)
blub = (np.array(list([list(obs_data['vind_value' + str(pred_window)]) for pred_window in range(0, 48, 1)])))
#shape of op (#ts, 48, 262)
op = np.swapaxes(blub, 0, 1)

#select of train, valid, test indicies
train_ind = [x for x in train_ind if x < op.shape[0]]
valid_ind = [x for x in valid_ind if x < op.shape[0]]
test_ind = [x for x in test_ind if x < op.shape[0]]

#split into train, vaild, test based on ts_ind
train_input = op[train_ind]
valid_input = op[valid_ind]
test_input = op[test_ind]

In [35]:
#this block contains instruction for to reduced data sets
factor_indices1 = [var_to_ind[x]- 1 for x in ["HR", "RR", "SBP", "DBP", "MBP", "O2 Saturation"]]
factor_indices2 = factor_indices1 + [V + x for x in factor_indices1] 

train_input = op[train_ind][:, :, factor_indices2]
valid_input = op[valid_ind][:, :, factor_indices2]
test_input = op[test_ind][:, :, factor_indices2] 

#change V for correct emedding generation
V = train_input.shape[2]//2

In [ ]:
with open('../gold_data/'+ dataset_name +'/gold_train.pickle', "wb") as outfile:
    pickle.dump(train_input, outfile, protocol = 4)
    
with open('../gold_data/'+ dataset_name +'/gold_valid.pickle', "wb") as outfile:
    pickle.dump(valid_input, outfile, protocol = 4)
    
with open('../gold_data/'+ dataset_name +'/gold_test.pickle', "wb") as outfile:
    pickle.dump(test_input, outfile, protocol = 4)    

# Generate embeddings

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchinfo import summary
from torch.optim.lr_scheduler import ReduceLROnPlateau

import importlib
import argparse
import sys

sys.path.append('LTSFLinear/models')
sys.path.append('LTSFLinear/')

import InformerAutoregressiveExtract as autoformer

embd_model = "/LTSFLinear/informer_mimic_reduced_3hours.pytorch"

In [ ]:
def set_seed(seed: int) -> None:
    """
    Set the random seed for modules torch, numpy and random.

    :param seed: random seed
    """
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    if torch.cuda.is_available() and torch.cuda.device_count() > 0:
        torch.backends.cudnn.deterministic = True
        torch.cuda.manual_seed_all(seed)
        
set_seed(42)

In [ ]:
parser = argparse.ArgumentParser(description='Autoformer & Transformer family for Time Series Forecasting')

# basic config
parser.add_argument('--is_training', type=int, required=False, default=1, help='status')
parser.add_argument('--train_only', type=bool, required=False, default=False, help='perform training on full input dataset without validation and testing')
parser.add_argument('--model_id', type=str, required=False, default='test', help='model id')
parser.add_argument('--model', type=str, required=False, default='Autoformer',
                    help='model name, options: [Autoformer, Informer, Transformer]')

# data loader
parser.add_argument('--data', type=str, required=False, default='ETTm1', help='dataset type')
parser.add_argument('--root_path', type=str, default='./data/ETT/', help='root path of the data file')
parser.add_argument('--data_path', type=str, default='ETTh1.csv', help='data file')
parser.add_argument('--features', type=str, default='M',
                    help='forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate')
parser.add_argument('--target', type=str, default='OT', help='target feature in S or MS task')
parser.add_argument('--freq', type=str, default='h',
                    help='freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h')
parser.add_argument('--checkpoints', type=str, default='./checkpoints/', help='location of model checkpoints')

# forecasting task
parser.add_argument('--seq_len', type=int, default=3, help='input sequence length')
parser.add_argument('--label_len', type=int, default=0, help='start token length')
parser.add_argument('--pred_len', type=int, default=3, help='prediction sequence length')


# DLinear
parser.add_argument('--individual', action='store_true', default=False, help='DLinear: a linear layer for each variate(channel) individually')
# Formers 
parser.add_argument('--embed_type', type=int, default=3, help='0: default 1: value embedding + temporal embedding + positional embedding 2: value embedding + temporal embedding 3: value embedding + positional embedding 4: value embedding')

#added for reduced data sets
parser.add_argument('--enc_in', type=int, default=V*2, help='encoder input size') # DLinear with --individual, use this hyperparameter as the number of channels
parser.add_argument('--dec_in', type=int, default=V, help='decoder input size')
parser.add_argument('--c_out', type=int, default=V, help='output size') 

parser.add_argument('--d_model', type=int, default=50, help='dimension of model')
parser.add_argument('--n_heads', type=int, default=8, help='num of heads')
parser.add_argument('--e_layers', type=int, default=2, help='num of encoder layers')
parser.add_argument('--d_layers', type=int, default=1, help='num of decoder layers')
parser.add_argument('--d_ff', type=int, default=2048, help='dimension of fcn')
parser.add_argument('--moving_avg', type=int, default=25, help='window size of moving average')
parser.add_argument('--factor', type=int, default=1, help='attn factor')
parser.add_argument('--distil', action='store_false',
                    help='whether to use distilling in encoder, using this argument means not using distilling',
                    default=False)
parser.add_argument('--dropout', type=float, default=0.05, help='dropout')
parser.add_argument('--embed', type=str, default='timeF',
                    help='time features encoding, options:[timeF, fixed, learned]')
parser.add_argument('--activation', type=str, default='gelu', help='activation')
parser.add_argument('--output_attention', action='store_true', help='whether to output attention in ecoder')
parser.add_argument('--do_predict', action='store_true', help='whether to predict unseen future data')

# optimization
parser.add_argument('--num_workers', type=int, default=10, help='data loader num workers')
parser.add_argument('--itr', type=int, default=2, help='experiments times')
parser.add_argument('--train_epochs', type=int, default=10, help='train epochs')
parser.add_argument('--batch_size', type=int, default=32, help='batch size of train input data')
parser.add_argument('--patience', type=int, default=3, help='early stopping patience')
parser.add_argument('--learning_rate', type=float, default=0.0001, help='optimizer learning rate')
parser.add_argument('--des', type=str, default='test', help='exp description')
parser.add_argument('--loss', type=str, default='mse', help='loss function')
parser.add_argument('--lradj', type=str, default='type1', help='adjust learning rate')
parser.add_argument('--use_amp', action='store_true', help='use automatic mixed precision training', default=False)

# GPU
parser.add_argument('--use_gpu', type=bool, default=True, help='use gpu')
parser.add_argument('--gpu', type=int, default=0, help='gpu')
parser.add_argument('--use_multi_gpu', action='store_true', help='use multiple gpus', default=False)
parser.add_argument('--devices', type=str, default='0,1,2,3', help='device ids of multile gpus')
parser.add_argument('--test_flop', action='store_true', default=False, help='See utils/tools for usage')

args = parser.parse_args(args=[])


In [ ]:
#initiate emedding model:
importlib.reload(autoformer)
batch_size, lr, patience = 32, 0.0005, 6  # batch_size increased, patience 10 --> 6
d, N, he, dropout = 50, 2, 4, 0.2
model = autoformer.Model(args).cuda()

# Load embedding model weights:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = ReduceLROnPlateau(optimizer, 'max', patience=3)
model.load_state_dict(torch.load(embd_model))

#put embedding model in eval mode:
model.eval()


data2embd = train_input #<-- train|valid|test_input
#funnel data through embedding model:
global_list1, global_list2 = [], []
for start in tqdm(range(0, len(data2embd), batch_size)):
    matrix = torch.tensor(train_input[start:start+batch_size], dtype=torch.float32).cuda() #MH: filter ts and cast to torch.tensor
    x = torch.tensor(matrix, dtype=torch.float32).cuda()
    liste = []
    liste2 = []
    for i in range(48//3):
        input_matrix = matrix[:, i*3:(i+1)*3] 
        input_mark = torch.arange(0, input_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
        input_mask = input_matrix[:, :24, V:]
        output_matrix = matrix[:, 24:, :V] 
        output_mark = torch.arange(0, output_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
        output_mask = matrix[:, 24:, V:]
        output_mark = torch.arange(0, output_matrix.size(1)).unsqueeze(0).repeat(input_matrix.size(0), 1).cuda()
        dec_inp = torch.zeros_like(output_matrix[:, -args.pred_len:, :]).float()   
        
        with torch.no_grad():
            output = model(input_matrix, input_mark, dec_inp, output_mark, trainn=False)
            output_real = output.mean(dim=1).unsqueeze(dim=1)
            liste.append(output_real)
            liste2.append(input_matrix.unsqueeze(1))
          
    outputs = torch.concat(liste, dim=1)
    inputs = torch.concat(liste2, dim=1)
    
    for mini in inputs:
        global_list1.append(mini)
        
    for mini in outputs:
        global_list2.append(mini)

In [ ]:
totallist = []

for inp, out in zip(global_list1, global_list2):
    totallist.append((inp.to('cpu').numpy(), out.to('cpu').numpy()))
 
#change outfile ending to '-<train|valid|test>.pickle' accordingly to data2embd
with open('../embeddings/embd3h_' + dataset_name + '-train.pickle', "wb") as outfile:
    pickle.dump(totallist, outfile, protocol = 4)